<a href="https://colab.research.google.com/github/Tafannum11259/Bookshop/blob/main/Music_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


In [ ]:
spotify = pd.read_csv("/content/drive/MyDrive/spotify.csv" , encoding = 'latin-1')
print(spotify.head())


NameError: name 'data' is not defined

In [ ]:
# Shape of DataSet
print(f"Number of column :{spotify.shape[1]}\nNumber of rows :{spotify.shape[0]}")

Number of column :24
Number of rows :953


In [ ]:
spotify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

In [ ]:
spotify.describe()

,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,in_apple_playlists,in_apple_charts,in_deezer_charts,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
count,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.00000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000
mean,1.556139,2018.238195,6.033578,13.930745,5200.124869,12.009444,67.812172,51.908709,2.666317,122.540399,66.96957,51.431270,64.279119,27.057712,1.581322,18.213012,10.131165
std,0.893044,11.116218,3.566435,9.201949,7897.608990,19.575992,86.441493,50.630241,6.035599,28.057802,14.63061,23.480632,16.550526,25.996077,8.409800,13.711223,9.912888
min,1.000000,1930.000000,1.000000,1.000000,31.000000,0.000000,0.000000,0.000000,0.000000,65.000000,23.00000,4.000000,9.000000,0.000000,0.000000,3.000000,2.000000
25%,1.000000,2020.000000,3.000000,6.000000,875.000000,0.000000,13.000000,7.000000,0.000000,100.000000,57.00000,32.000000,53.000000,6.000000,0.000000,10.000000,4.000000
50%,1.000000,2022.000000,6.000000,13.000000,2224.000000,3.000000,34.000000,38.000000,0.000000,121.000000,69.00000,51.000000,66.000000,18.000000,0.000000,12.000000,6.000000
75%,2.000000,2022.000000,9.000000,22.000000,5542.000000,16.000000,88.000000,87.000000,2.000000,140.000000,78.00000,70.000000,77.000000,43.000000,0.000000,24.000000,11.000000
max,8.000000,2023.000000,12.000000,31.000000,52898.000000,147.000000,672.000000,275.000000,58.000000,206.000000,96.00000,97.000000,97.000000,97.000000,91.000000,97.000000,64.000000


In [ ]:
print(f"number of artists '{spotify['artist(s)_name'].nunique()}'")

number of artists '645'


In [ ]:
artist_counts = spotify['artist(s)_name'].value_counts()
artist_counts

,count
artist(s)_name,
Taylor Swift,34
The Weeknd,22
Bad Bunny,19
SZA,19
Harry Styles,17
...,...
"Karol G, Ovy On The Drums",1
"Coolio, L.V.",1
Kordhell,1


### **Visualization**

In [ ]:
iplot(px.bar(artist_counts[:15],
             text_auto = True,
             color = artist_counts[:15].index,
             color_discrete_sequence = colors,
             labels=dict(index="Count Names",value="Count"),
             title = 'Top 15 Artists with Most Songs'
            ))

### **Random Forest**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Convert columns to numeric, setting errors='coerce' to replace non-numeric values with NaN
for col in features:
    spotify[col] = pd.to_numeric(spotify[col], errors='coerce')

# Define features and target
X = spotify[features]
y = spotify['repeat_play']

# Handle missing values and scale features
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X = imputer.fit_transform(X)
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print accuracy
print(f"Accuracy: {accuracy:.2f}")

# Print classification report in a more readable format
print("\nClassification Report:\n")
print(classification_rep)



Accuracy: 0.84

Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.86      0.85       150
           1       0.84      0.82      0.83       136

    accuracy                           0.84       286
   macro avg       0.84      0.84      0.84       286
weighted avg       0.84      0.84      0.84       286



**Logistic Regression**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
import pandas as pd

# Convert columns to numeric, setting errors='coerce' to replace non-numeric values with NaN
for col in features:
    spotify[col] = pd.to_numeric(spotify[col], errors='coerce')

# Define features and target
X = spotify[features]
y = spotify['repeat_play']

# Handle missing values and scale features
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X = imputer.fit_transform(X)
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the LogisticRegression model
model = LogisticRegression(random_state=42, max_iter=1000)  # Set max_iter to ensure convergence
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print accuracy
print(f"Accuracy: {accuracy:.2f}")

# Print classification report in a more readable format
print("\nClassification Report:\n")
print(classification_rep)


Accuracy: 0.86

Classification Report:

              precision    recall  f1-score   support

           0       0.83      0.93      0.88       150
           1       0.91      0.79      0.85       136

    accuracy                           0.86       286
   macro avg       0.87      0.86      0.86       286
weighted avg       0.87      0.86      0.86       286

